# RFM 분석

- RFM 분석은 고객 세분화를 위해 효과적인 마케팅 분석 도구
- R(Recency 최근성), F(Frequency 빈도), M(Monetary 금액)의 약자
  -  R(Recency 최근성) : (일정기간동안) 고객이 얼마나 최근에 구매(방문,접속) 했는가
  -  F(Frequency 빈도) : (일정기간동안) 고객이 얼마나 자주 구매적(방문,접속) 했는가
  -  M(Monetary 금액) : (일정기간동안) 고객이 얼마나 구매했는가
- R/M/F 값을 계산하여 합산 한 다음 최종 합산 점수로 고객을 분할
- SCORE = 0.2*R + 0.3*M + 0.5*F (가중치는 실무자들의 회의를 통해 결정)

In [2]:
import pandas as pd

In [6]:
# Encoding : 컴퓨터가 알아들을 수 있는 형테로 문자를 변환하는 작업
# 간혹 데이터를 불러올 때, 한글 서체에 대한 인코딩 정보가 들어있지 않는 경우 (UnicodeDecodeError)

df1 = pd.read_csv('data\\09_Data.csv',encoding = 'cp949',index_col=0)
print(df1.shape)
df1.info()

(46749, 19)
<class 'pandas.core.frame.DataFrame'>
Index: 46749 entries, 0 to 46748
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   결제수단        46749 non-null  object 
 1   교환구분        46749 non-null  object 
 2   배송번호        46749 non-null  object 
 3   배송시작일       46749 non-null  object 
 4   배송완료일       46749 non-null  object 
 5   사용한 적립금액    46749 non-null  int64  
 6   상품구매금액      46749 non-null  int64  
 7   상품명         46661 non-null  object 
 8   상품번호        46661 non-null  float64
 9   상품별 추가할인금액  46749 non-null  int64  
 10  수량          46749 non-null  int64  
 11  주문 시 회원등급   43644 non-null  object 
 12  주문경로        46749 non-null  object 
 13  주문번호        46749 non-null  object 
 14  주문일시        46749 non-null  object 
 15  주문자ID       46749 non-null  object 
 16  카드사         16611 non-null  object 
 17  쿠폰 할인금액     46749 non-null  int64  
 18  품목번호        46749 non-null  int64  
dtypes: float64(1), int

In [7]:
df1.head(5)

,결제수단,교환구분,배송번호,배송시작일,배송완료일,사용한 적립금액,상품구매금액,상품명,상품번호,상품별 추가할인금액,수량,주문 시 회원등급,주문경로,주문번호,주문일시,주문자ID,카드사,쿠폰 할인금액,품목번호
0,무통장입금,교환안함,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,71450,195파이 탕용기 소 300개 1set(2박스),1077.0,13600,1,전화주문 고객,PC쇼핑몰,20181227-0000648,2018-12-27 오후 2:54,0V00A1,NaN,0,42602
1,무통장입금,교환안함,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,141240,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42.0,19800,2,일반,모바일웹,20181229-0000119,2018-12-29 오후 1:22,1V00A1,NaN,0,42930
2,무통장입금,교환안함,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,13910,비닐봉투 배달중 중 백색 200개,1271.0,2900,1,NaN,모바일웹,20181230-0000100,2018-12-30 오후 3:15,2V00A1,NaN,0,42998
3,무통장입금,교환안함,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,81600,195파이 미니탕용기 투톤 300개 1set(2박스),1085.0,12630,1,일반,모바일웹,20181231-0000087,2018-12-31 오전 10:30,3V00A1,NaN,0,43063
4,무통장입금,교환안함,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,94710,250파이 원형 접시 미니 블랙 400개 1박스,1106.0,18000,1,사업자,모바일웹,20181231-0000108,2018-12-31 오전 10:43,4V00A1,NaN,0,43066


In [14]:
# 날짜데이터 처리
def func1(row) :
    return row.split(' ')[0]
    
df1['주문일자'] = pd.to_datetime(df1['주문일시'].apply(func1))

In [15]:
# 최근성 계산
df1['Renc'] = max(df1['주문일자']) - df1['주문일자']

In [21]:
# 고객별 최근성 계산
p0 = df1.pivot_table(index='주문자ID', values='Renc' , aggfunc = 'min').reset_index()
p0['Renc']

0       30 days
1       59 days
2       30 days
3      142 days
4      142 days
         ...   
4952   112 days
4953   142 days
4954    22 days
4955    34 days
4956   100 days
Name: Renc, Length: 4957, dtype: timedelta64[ns]

In [26]:
def func2(row) :
    return int(str(row).split(' ')[0])

p0['Recency'] = p0['Renc'].apply(func2)
p0['Recency'].describe()

count    4957.000000
mean      186.366149
std       173.458076
min         0.000000
25%        47.000000
50%       121.000000
75%       287.000000
max       899.000000
Name: Recency, dtype: float64

In [25]:
# 빈도계산
df1['Frequency'] = 1
p1 = df1.pivot_table(index=['주문자ID','주문일자'], values='Frequency' , aggfunc='min').reset_index()
p2 = p1.pivot_table(index='주문자ID', values='Frequency', aggfunc='sum').reset_index()
p2

,주문자ID,Frequency
0,0V00A1,6
1,1000V00A1,12
2,1001V00A1,3
3,1002V00A1,1
4,1003V00A1,1
...,...,...
4952,997V00A1,5
4953,998V00A1,1
4954,999V00A1,2
4955,99V00A1,11


In [27]:
# 금액계산
p3 = df1.pivot_table(index='주문자ID', values='상품구매금액', aggfunc='sum').reset_index()
p4 = p3.rename(columns={'상품구매금액' : 'Monetary'})

In [28]:
# 병합
m1 = pd.merge(p0,p2, on='주문자ID', how='inner')
m2 = pd.merge(m1,p4, on='주문자ID', how='inner')

In [30]:
# 고객들의 개인정보
df2 = pd.read_csv(r'C:\Users\UserK\Desktop\Ranee\data\ML\10_Data.csv')